$$
    \newcommand{\prover}{\mathcal{P}}
    \newcommand{\verifier}{\mathcal{V}}
    \newcommand{\Mat}[1]{\mathbf{{#1}}}
$$

# Spartan: Efficient and general-purpose zkSNARKs without trusted setup

Authors: Srinath Setty <br/>
ePrint: [2019/550](https://eprint.iacr.org/2019/550.pdf)

## Introduction

Spartan is a transparent [SNARK](https://en.wikipedia.org/wiki/Non-interactive_zero-knowledge_proof) for R1CS [constraint satisfaction problem](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem). R1CS is a popular NP-Complete problem that enjoys extensive tooling support from compiler community. 

An R1CS instance consists of
1. Three $n \times m$ _sparse_ matrices $A, B, C \in \mathbb{F}^{n\times m}$ that encode the circuit/constraints representing the CSP, and 
2. An arbitrary $m$ dimensional vector $\vec{w} \in \mathbb{F}^m$.

The vector $\vec{w}$ is _considered a witness_ to the NP statement if it satisfies the following relation:

$$
    (A\cdot \vec{w}) {\color{red} \circ }(B\cdot \vec{w}) = C\cdot \vec{w}
$$

where ${\color{red} \circ }$ is the Hadamard (element wise) product.

Let $s = \max\{\log_2(m), \log_2(n) \}$ and let $\ell$ be the maximum number of non-zero elements in $A, B,$ or $C$ matrices. For matrices $A, B,$ or $C$ to be considered sparse, we further assume that $\ell \ll 2^s$.

Spartan is a multivariate SNARK where 
* the prover's time and space requirement is _strictly linear_ in $\ell$, and 
* the verifier time is $O(s)$, (i.e., polylog in $\ell$). 

Note that in a typical use case (e.g. google's [zk-longfellow](https://github.com/google/longfellow-zk.git) for legacy identity verification), the value of $\ell$ is of the order of $2^{25}$. Unless the prover time is linear (or at worse quasi-linear) in $\ell$, it will be hard to build a practical system using current hardware.  

### Spartan Attempt-0

![Spartan Attempt 0](./spartan/Spartan-0.drawio.svg)

### Spartan Full
![Spartan Sumcheck-1](./spartan/Spartan-Full-SCHK-1.drawio.svg)

![Spartan Sumcheck-2](./spartan/Spartan-Full-SCHK-2.drawio.svg)

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('./sage-snark'))
sys.path.insert(0, module_path)

In [2]:
from sage.matrix.all import Matrix
from sage.rings.integer import Integer
from sage.rings.polynomial.all import PolynomialRing, Polynomial
from sage.modules.all import vector

from utils.multivariates import hadamard_product, hypercube_sum, bit_decomp_dict, matrix_multilinearize, vec_multilinearize
from utils.test_utils import R1CS

In [ ]:
# Sample R1CS Instance. Taken from: https://emirsoyturk.medium.com/hello-arithmetization-55e57c8e5471

AL = [
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 1, 0, 0, 1, 0],
     [5, 0, 0, 0, 0, 1]
    ]

BL = [
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0]
    ]

CL = [
     [0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1],
     [0, 0, 1, 0, 0, 0]
    ]

WL = [1, 3, 35, 9, 27, 30]

In [ ]:
def compute_y_sum(poly, y_dim):
    gens = poly.parent().gens()
    skip_len = len(gens) - Integer(y_dim).bit_length()
    assert skip_len >= 0

    skip_vars = gens[:skip_len]
    return hypercube_sum(poly, skip_vars)

In [ ]:
P = 15*(2**27) + 1;
assert is_prime(P)
Fp = GF(P)

A = Matrix(Fp, AL)
B = Matrix(Fp, BL)
C = Matrix(Fp, CL)
w = vector(Fp, WL);
aw = A*w;
bw = B*w;
cw = C*w;

# print(f"W = {w}")
# print(f"A*W = {aw}")
# print(f"B*W = {bw}")
# print(f"C*W = {cw}")

assert list(cw) == hadamard_product(aw, bw)

In [ ]:
Axy = matrix_multilinearize(A);
Bxy = matrix_multilinearize(B, Axy.parent())
Cxy = matrix_multilinearize(C, Axy.parent())

xgens_count = Integer(A.nrows()).bit_length()
xgens = Axy.parent().gens()[:xgens_count];
ygens = Axy.parent().gens()[xgens_count:];

Wy = vec_multilinearize(w, ygens)

Ax = compute_y_sum(Axy*Wy, 6)
Bx = compute_y_sum(Bxy*Wy, 6)
Cx = compute_y_sum(Cxy*Wy, 6)

expected = Ax*Bx - Cx

for i in range(A.nrows()):
    d = bit_decomp_dict(i, xgens)
    e = expected.subs(d)
    assert e == 0
